In [ ]:
# Import reqired libraries
import numpy as np
import yfinance as yf
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import os
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.stats import gaussian_kde
from matplotlib.backends.backend_pdf import PdfPages
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Set display options
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.width', 1000)        # Set display width to a high value

In [ ]:
# Function to print colored text
def print_colored(text, color):
    color_codes = {
        'black': '0;30',
        'red': '0;31',
        'green': '0;32',
        'yellow': '0;33',
        'blue': '0;34',
        'purple': '0;35',
        'cyan': '0;36',
        'white': '0;37',
        'bold_black': '1;30',
        'bold_red': '1;31',
        'bold_green': '1;32',
        'bold_yellow': '1;33',
        'bold_blue': '1;34',
        'bold_purple': '1;35',
        'bold_cyan': '1;36',
        'bold_white': '1;37'
    }
    color_code = color_codes.get(color, '0;30')  # Default to white if color not found
    print(f"\033[{color_code}m{text}\033[0m")

nice_fonts = {
        # Use LaTeX to write all text
        "text.usetex": True,
        "font.family": "serif",
        # Use 10pt font in plots, to match 10pt font in document
        "axes.labelsize": 10,
        "font.size": 10,
        # Make the legend/label fonts a little smaller
        "legend.fontsize": 10,
        "xtick.labelsize": 10,
        "ytick.labelsize": 10,
        #'text.latex.preamble' : [r'\usepackage{amsmath}'],
        'mathtext.fontset' : 'stix',
        'mathtext.rm' : 'serif'
}
mpl.rcParams.update(nice_fonts)

In [ ]:
# Load the dataset
df = pd.read_csv('Sweden_data_processed.csv')

In [ ]:
# Function to perform EDA for a given asset
def perform_eda(asset, df):
    print_colored(f"--- Analysis for {asset} ---", "blue")
    
    # Decompose the time series for trend analysis
    decomposition = seasonal_decompose(df['Close'], model='additive', period=365)
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid

    # Create a PDF file to save the plots
    with PdfPages(f'../plots/{asset}_analysis.pdf') as pdf:
    
        # Create a large plot with subplots for various analyses
        fig, axes = plt.subplots(7, 2, figsize=(18, 28), sharex=False)
        #fig.suptitle(f'Analysis for {asset}', fontsize=16)
        
        # Time series of closing price
        axes[0, 0].plot(df.index, df['Close'], label='Close Price')
        axes[0, 0].set_ylabel('Price')
        axes[0, 0].set_xlabel('Year')
        axes[0, 0].legend()
        axes[0, 0].set_xlim(df.index.min(), df.index.max())
        axes[0, 0].set_ylim(df['Close'].min(), df['Close'].max())
        
        # Time series of volume
        axes[0, 1].plot(df.index, df['Volume'], label='Volume', color='orange')
        axes[0, 1].set_ylabel('Volume')
        axes[0, 1].set_xlabel('Year')
        axes[0, 1].legend()
        axes[0, 1].set_xlim(df.index.min(), df.index.max())
        axes[0, 1].set_ylim(df['Volume'].min(), df['Volume'].max())
        
        # Time series of moving averages
        axes[1, 0].plot(df.index, df['MA20'], label='MA20', color='green')
        axes[1, 0].plot(df.index, df['MA50'], label='MA50', color='red')
        axes[1, 0].set_ylabel('Moving Averages')
        axes[1, 0].set_xlabel('Year')
        axes[1, 0].legend()
        axes[1, 0].set_xlim(df.index.min(), df.index.max())
        axes[1, 0].set_ylim(min(df['MA20'].min(), df['MA50'].min()), max(df['MA20'].max(), df['MA50'].max()))
        
        # Time series of exponential moving averages
        axes[1, 1].plot(df.index, df['EMA20'], label='EMA20', color='blue')
        axes[1, 1].plot(df.index, df['EMA50'], label='EMA50', color='purple')
        axes[1, 1].set_ylabel('Exponential Moving Averages')
        axes[1, 1].set_xlabel('Year')
        axes[1, 1].legend()
        axes[1, 1].set_xlim(df.index.min(), df.index.max())
        axes[1, 1].set_ylim(min(df['EMA20'].min(), df['EMA50'].min()), max(df['EMA20'].max(), df['EMA50'].max()))
        
        # Time series of volatility
        axes[2, 0].plot(df.index, df['Volatility'], label='Volatility', color='purple')
        axes[2, 0].set_ylabel('Volatility')
        axes[2, 0].set_xlabel('Year')
        axes[2, 0].legend()
        axes[2, 0].set_xlim(df.index.min(), df.index.max())
        axes[2, 0].set_ylim(df['Volatility'].min(), df['Volatility'].max())
        
        # Time series of RSI
        axes[2, 1].plot(df.index, df['RSI'], label='RSI', color='brown')
        axes[2, 1].set_ylabel('RSI')
        axes[2, 1].set_xlabel('Year')
        axes[2, 1].legend()
        axes[2, 1].set_xlim(df.index.min(), df.index.max())
        axes[2, 1].set_ylim(0, 100)
        
        # MACD and Signal Line
        axes[3, 0].plot(df.index, df['MACD'], label='MACD', color='blue')
        axes[3, 0].plot(df.index, df['Signal Line'], label='Signal Line', color='red')
        axes[3, 0].set_ylabel('MACD')
        axes[3, 0].set_xlabel('Year')
        axes[3, 0].legend()
        axes[3, 0].set_xlim(df.index.min(), df.index.max())
        
        # Daily return distribution
        data = df['Daily Return']
        n, bins, patches = axes[3, 1].hist(data, bins=50, density=True, alpha=0.7, color='steelblue', edgecolor='black', label='Daily Returns Distribution')
        kde = gaussian_kde(data)
        x = np.linspace(data.min(), data.max(), 1000)
        axes[3, 1].plot(x, kde(x), color='blue', linestyle='--', label='KDE')
        axes[3, 1].legend()
        axes[3, 1].set_xlabel('Daily Return')
        axes[3, 1].set_ylabel('Frequency')
        axes[3, 1].set_xlim(data.min(), data.max())
        
        # Cumulative daily return
        axes[4, 0].plot(df.index, df['Daily Return'].cumsum(), label='Cumulative Daily Return')
        axes[4, 0].set_xlabel('Year')
        axes[4, 0].set_ylabel('Cumulative Return')
        axes[4, 0].legend()
        axes[4, 0].set_xlim(df.index.min(), df.index.max())
        axes[4, 0].set_ylim(df['Daily Return'].cumsum().min(), df['Daily Return'].cumsum().max())
        
        # Trend analysis observed
        axes[4, 1].plot(df.index, df['Close'], label='Observed')
        axes[4, 1].set_xlabel('Year')
        axes[4, 1].set_ylabel('Price')
        axes[4, 1].legend()
        axes[4, 1].set_xlim(df.index.min(), df.index.max())
        axes[4, 1].set_ylim(df['Close'].min(), df['Close'].max())
        
        # Trend analysis
        axes[5, 0].plot(df.index, trend, label='Trend')
        axes[5, 0].set_xlabel('Year')
        axes[5, 0].set_ylabel('Price')
        axes[5, 0].legend()
        axes[5, 0].set_xlim(df.index.min(), df.index.max())
        axes[5, 0].set_ylim(trend.min(), trend.max())
        
        # Trend analysis seasonal
        axes[5, 1].plot(df.index, seasonal, label='Seasonal')
        axes[5, 1].set_xlabel('Year')
        axes[5, 1].set_ylabel('Price')
        axes[5, 1].legend()
        axes[5, 1].set_xlim(df.index.min(), df.index.max())
        axes[5, 1].set_ylim(seasonal.min(), seasonal.max())
        
        # Trend analysis residual
        axes[6, 0].plot(df.index, residual, label='Residual')
        axes[6, 0].set_xlabel('Year')
        axes[6, 0].set_ylabel('Price')
        axes[6, 0].legend()
        axes[6, 0].set_xlim(df.index.min(), df.index.max())
        axes[6, 0].set_ylim(residual.min(), residual.max())
        
        # Daily return time series
        axes[6, 1].plot(df.index, df['Daily Return'], label='Daily Return', color='blue')
        axes[6, 1].set_ylabel('Daily Return')
        axes[6, 1].set_xlabel('Year')
        axes[6, 1].legend()
        axes[6, 1].set_xlim(df.index.min(), df.index.max())
        axes[6, 1].set_ylim(df['Daily Return'].min(), df['Daily Return'].max())

        fig.subplots_adjust(left=0.04,
                    bottom=0.02, 
                    right=0.98, 
                    top=0.99, 
                    wspace=0.2, 
                    hspace=0.3)

        # Save the figure to the PDF and show it
        pdf.savefig(fig)
        plt.show()
        plt.close(fig)

        # Correlation matrix
        fig_corr, ax_corr = plt.subplots(figsize=(12, 9))
        corr = df.corr()
        sns.heatmap(corr, annot=True, cmap='coolwarm', linewidths=0.5, ax=ax_corr, annot_kws={"size": 12})
        ax_corr.set_title('Correlation Matrix')
        ax_corr.set_aspect(aspect='equal', adjustable='box')
        fig_corr.tight_layout()
        
        # Save the figure to the PDF and show it
        pdf.savefig(fig_corr)
        plt.show()
        plt.close(fig_corr)


In [ ]:


# Perform EDA for each asset
for asset, df in data.items():
  perform_eda(asset, df)